In [162]:
import sqlite3
import csv
import pandas as pd
from datetime import datetime

In [163]:
def read_sales_csv(filename):
    sales = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            date = datetime.strptime(row['date'], '%m/%d/%y')
            sales.append({'date': date,
                     'days_ago': (datetime.now() - date).days,
                     'account_number': row['account_number'],
                     'price': int(row['price'])})
    return sales

conn = sqlite3.connect('file:NW_Central_OKC_w_bldg_details.db?mode=ro', uri=True)
c = conn.cursor()

sales = read_sales_csv('sales_list_2019.csv')
#sales = sales + read_sales_csv('sales_list_2018.csv')
#sales = sales + read_sales_csv('sales_list_2017.csv')

Now let's load the data into DataFrames. We have two tables we're using, one for properties/parcels and one for buildings. So what we'll do is pull out some data about the buildings, sum it up (as some properties have more than 1 building), and add it to the property DF.

In [164]:
combined_df = pd.read_sql_query("SELECT * FROM realproperty INNER JOIN buildings ON realproperty.id = buildings.local_property_id;", conn)
rp_df = pd.read_sql_query("SELECT * FROM realproperty WHERE property_type = 'Residential'", conn)
rp_df = rp_df.set_index('account_number')

# Get data about the buildings for each parcel. We'll sum up the data for now
# (in cases where there's >1 building on the property)
rp_df['sqft_sum'] = combined_df.groupby(['account_number'])['sq_ft'].sum()
rp_df['bed_sum'] = combined_df.replace(-1,0).groupby(['account_number'])['bedrooms'].sum()
rp_df['bath_sum'] = combined_df.replace(-1,0).groupby(['account_number'])['full_bathrooms'].sum() +\
    (combined_df.replace(-1,0).groupby(['account_number'])['three_quarters_bathrooms'].sum() * 0.75) +\
    (combined_df.replace(-1,0).groupby(['account_number'])['half_bathrooms'].sum() * 0.5)
rp_df['room_sum'] = combined_df.replace(-1,0).groupby(['account_number'])['total_rooms'].sum()
rp_df['main_bldg_sqft'] = combined_df.groupby(['account_number'])['sq_ft'].max()
rp_df['year_built'] = combined_df[combined_df.year_built != -1].groupby(['account_number'])['year_built'].mean()
rp_df['bldg_age'] = pd.datetime.now().year - rp_df['year_built']

display(rp_df)

,id,propertyid,property_type,location,building_name_occupant,city,owner_name_1,owner_name_2,billing_address_1,billing_address_2,city_state_zip,quarter_section,parent_acct,tax_district,school_system,land_size,lot_width,lot_depth,land_value,quarter_section_description,subdivision,block,lot,legal_description,sqft_sum,bed_sum,bath_sum,room_sum,main_bldg_sqft,year_built,bldg_age
account_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
R049810025,1,130116,Residential,400 NW 43RD ST,,OKLAHOMA CITY,YOUNGS STEVEN ROGERS,TURLEY ALLISON,400 NW 43RD ST,,"OKLAHOMA CITY, OK 73118-8221",2661,,TXD 200,Oklahoma City #89,13068.0,None,None,52721,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,001,EDGEMERE TERRACE 001 001,1824.0,3.0,2.00,7.0,1824.0,1951.0,68.0
R049810050,3,130117,Residential,408 NW 43RD ST,,OKLAHOMA CITY,JUNGMAN TIMOTHY WILLIAM & JULIE LYN,,408 NW 43RD ST,,"OKLAHOMA CITY, OK 73118-8221",2661,,TXD 200,Oklahoma City #89,9147.6,None,None,45900,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,002,EDGEMERE TERRACE 001 002,2677.0,3.0,2.50,6.0,2677.0,1951.0,68.0
R049480080,6,130504,Residential,1009 NW 42ND ST,,OKLAHOMA CITY,SPEARS KENNETH L & REGINA,,210 NW 10TH ST,,"OKLAHOMA CITY, OK 73103-3902",2663,,TXD 200,Oklahoma City #89,7405.2,None,None,71460,Sect 16-T12N-R3W Qtr SW,CROWN HEIGHTS ADD,001,004,CROWN HEIGHTS ADD 001 004,1961.0,2.0,2.00,9.0,1961.0,1939.0,80.0
R049640005,7,131004,Residential,5021 N WALKER AVE,,OKLAHOMA CITY,CRISP STEVE & ANGIE,,5021 N WALKER,,"OKLAHOMA CITY, OK 73118",2664,,TXD 200,Oklahoma City #89,9147.6,None,None,48230,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 1 & 2,1352.0,3.0,1.50,6.0,1352.0,1951.0,68.0
R049640010,8,131005,Residential,504 NW 50TH ST,,OKLAHOMA CITY,DICKSON JERRY ROBERT,,504 NW 50TH ST,,"OKLAHOMA CITY, OK 73118-6204",2664,,TXD 200,Oklahoma City #89,6969.6,None,None,37100,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 3 & 4,1386.0,3.0,1.50,6.0,1386.0,1951.0,68.0
R049640015,9,131006,Residential,508 NW 50TH ST,,OKLAHOMA CITY,GARRETT & COMPANY LLC,,9701 BROADWAY EXT,,"OKLAHOMA CITY, OK 73114-6316",2664,,TXD 200,Oklahoma City #89,6969.6,None,None,37100,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 5 & 6,1092.0,2.0,1.00,4.0,1092.0,1950.0,69.0
R049810100,10,130119,Residential,418 NW 43RD ST,,OKLAHOMA CITY,BRADEN THOMAS GARY & BEVERLY JOAN,REVOCABLE LIVING TRUST,418 NW 43RD ST,,"OKLAHOMA CITY, OK 73118-8221",2661,,TXD 200,Oklahoma City #89,9147.6,None,None,45900,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,004,EDGEMERE TERRACE 001 004,2047.0,2.0,2.00,7.0,2047.0,1951.0,68.0
R049640020,12,131007,Residential,512 NW 50TH ST,,OKLAHOMA CITY,HALPIN JEFFREY J,,512 NW 50TH ST,,"OKLAHOMA CITY, OK 73118-6204",2664,,TXD 200,Oklahoma City #89,6969.6,None,None,37100,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 7 & 8,1038.0,2.0,1.00,4.0,1038.0,1950.0,69.0
R049480100,13,130505,Residential,1005 NW 42ND ST,,OKLAHOMA CITY,DK INC,,C/O KENNETH L SPEARS,PO BOX 687,"OKLAHOMA CITY, OK 73101",2663,,TXD 200,Oklahoma City #89,7405.2,None,None,71460,Sect 16-T12N-R3W Qtr SW,CROWN HEIGHTS ADD,001,005,CROWN HEIGHTS ADD 001 005,1660.0,2.0,2.00,8.0,1660.0,1947.0,72.0


Now let's add more features for the main building. To do this we'll create a new DataFrame containing the features of the largest building on each property. While most single-family properties only have one building, we do need to consider that some will have multiple.

In [165]:
# we want 26584 not 26631
idx = combined_df.groupby(['account_number'])['sq_ft'].transform(max) == combined_df['sq_ft']
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 100)
combined_df2 = combined_df[idx]


# now that we have the ones with largest sq ft on the parcel, let's weed out duplicates (where there are
# multiple bldgs with identical sq ft)
idx = combined_df2.groupby(['account_number'])['bldg_id'].transform(max) == combined_df2['bldg_id']
combined_df2 = combined_df2[idx]
combined_df2 = combined_df2.set_index(['account_number'])
combined_df2

,id,propertyid,property_type,location,building_name_occupant,city,owner_name_1,owner_name_2,billing_address_1,billing_address_2,city_state_zip,quarter_section,parent_acct,tax_district,school_system,land_size,lot_width,lot_depth,land_value,quarter_section_description,subdivision,block,lot,legal_description,id,local_property_id,bldg_id,vacant_or_improved,bldg_description,year_built,sq_ft,number_stories,remodel_year,building_name,alt_land_use_desc,quality_description,frame_description,foundation_description,exterior,roof_type,roof_cover,avg_floor_height,percent_sprinkled,total_rooms,bedrooms,full_bathrooms,three_quarters_bathrooms,half_bathrooms,hvac_type,physical_condition,number_res_units,number_comm_units
account_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
R049810025,1,130116,Residential,400 NW 43RD ST,,OKLAHOMA CITY,YOUNGS STEVEN ROGERS,TURLEY ALLISON,400 NW 43RD ST,,"OKLAHOMA CITY, OK 73118-8221",2661,,TXD 200,Oklahoma City #89,13068.0,None,None,52721,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,001,EDGEMERE TERRACE 001 001,1,1,1,Improved,Ranch 1 Story,1951,1824,1.0,2014,,Residential Improvement,Fair,,Conventional,Frame Masonry Veneer,Hip,Composition Shingle,8,0,7,3,2,0,0,Central Air to Air,Average,1,0
R049480020,2,130501,Commercial,4300 N WESTERN AVE,PARKING FOR BARREL AND DRUM ROOM,OKLAHOMA CITY,VHG PROPERTY INTERESTS LLC,,PO BOX 18877,,"OKLAHOMA CITY, OK 73154",2663,,TXD 200,Oklahoma City #89,10454.4,None,None,124585,Sect 16-T12N-R3W Qtr SW,CROWN HEIGHTS ADD,001,000,CROWN HEIGHTS ADD 001 000 S89.5FT LOTS 1 & 2,2,2,1,Improved,Parking Lot,2005,0,1.0,-1,PARKING FOR BARREL AND DRUM ROOM,Commercial Improvement,None,None,,,,,8,0,-1,-1,0,0,0,None,Poor,0,0
R049810050,3,130117,Residential,408 NW 43RD ST,,OKLAHOMA CITY,JUNGMAN TIMOTHY WILLIAM & JULIE LYN,,408 NW 43RD ST,,"OKLAHOMA CITY, OK 73118-8221",2661,,TXD 200,Oklahoma City #89,9147.6,None,None,45900,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,002,EDGEMERE TERRACE 001 002,3,3,1,Improved,Split Level,1951,2677,1.0,2010,,Residential Improvement,Fair,,Conventional,Frame Masonry Veneer,Hip,Composition Shingle,8,0,6,3,2,0,1,Central Air to Air,Average,1,0
R049480040,4,130502,Commercial,4308 N WESTERN AVE,BARREL,OKLAHOMA CITY,VHG PROPERTY INTERESTS LLC,,PO BOX 18877,,"OKLAHOMA CITY, OK 73154-0877",2663,,TXD 200,Oklahoma City #89,13068.0,None,None,79000,Sect 16-T12N-R3W Qtr SW,CROWN HEIGHTS ADD,001,000,CROWN HEIGHTS ADD 001 000 N50FT LOTS 1 & 2 & A...,4,4,1,Improved,Restaurant,1935,4738,1.0,2005,BARREL,Commercial Improvement,Fair,Masonry,,,Flat,,12,0,-1,-1,0,0,0,Complete HVAC,Good,0,1
R056050500,5,131543,Commercial,1112 NW 50TH ST,QUEEN BEE/ PRAIRIE ARTS/ BOARD UP,OKLAHOMA CITY,FOREST SIDING SUPPLY LLC,,1117 NW 50TH ST,,"OKLAHOMA CITY, OK 73118",2665,,TXD 200,Oklahoma City #89,15681.6,None,None,78000,Sect 17-T12N-R3W Qtr NE,ISLE VIEW SUB ADD,001,000,ISLE VIEW SUB ADD 001 000 LOTS 7 THRU 10,5,5,1,Improved,Ranch 1 Story,1925,1708,1.0,2000,QUEEN BEE/ PRAIRIE ARTS/ BOARD UP,Commercial Improvement,Average,,Slab,Frame Masonry Veneer,Gable,Composition Shingle,8,0,0,0,0,0,0,Central Air to Air,Average,1,0
R049480080,6,130504,Residential,1009 NW 42ND ST,,OKLAHOMA CITY,SPEARS KENNETH L & REGINA,,210 NW 10TH ST,,"OKLAHOMA CITY, OK 73103-3902",2663,,TXD 200,Oklahoma City #89,7405.2,None,None,71460,Sect 16-T12N-R3W Qtr SW,CROWN HEIGHTS ADD,001,004,CROWN HEIGHTS ADD 001 004,7,6,1,Improved,Duplex One Story,1939,1961,1.0,1977,,Residential Improvement,Fair,,Conventional,Frame Masonry Veneer,Hip,Composition Shingle,8,0,9,2,2,0,0,Central Air to Air,Average,2,0
R049640005,7,131004,Residential,5021 N WALKER AVE,,OKLAHOMA CITY,CRISP STEVE & ANGIE,,5021 N WALKER,,"OKLAHOMA CITY, OK 73118",2664,,TXD 200,Oklahoma City #89,9147.6,None,None,48230,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 1 & 2,8,7,1,Improved,Ranch 1 Story,1951,1352,1.0,1997,,Residential Improvement,Fair,,Conventional,Frame Masonry Veneer,Gable,Composition Shingle,8,0,6,3,1,0,1,Central Air to 

Now we have the features for the largest building on each parcel. Let's add some of these to combined_df (like we did above).

In [166]:
print(len(rp_df))
print(len(combined_df2))
#print(combined_df2.groupby(['account_number'])['bldg_description'])
rp_df['main_bldg_description'] = combined_df2['bldg_description']
rp_df['main_bldg_hvac'] = combined_df2['hvac_type']
rp_df['main_bldg_quality'] = combined_df2['quality_description']
# Floor height is not worth using as it is listed as 8 on the vast majority of residential properties
rp_df['main_bldg_exterior'] = combined_df2['exterior']
rp_df['main_bldg_roof'] = combined_df2['roof_type']
rp_df['main_bldg_year'] = combined_df2['year_built']
rp_df['main_bldg_effective_year'] = combined_df2[['year_built', 'remodel_year']].max(axis=1) # remodel year, else built year
rp_df

21978
26584


,id,propertyid,property_type,location,building_name_occupant,city,owner_name_1,owner_name_2,billing_address_1,billing_address_2,city_state_zip,quarter_section,parent_acct,tax_district,school_system,land_size,lot_width,lot_depth,land_value,quarter_section_description,subdivision,block,lot,legal_description,sqft_sum,bed_sum,bath_sum,room_sum,main_bldg_sqft,year_built,bldg_age,main_bldg_description,main_bldg_hvac,main_bldg_quality,main_bldg_exterior,main_bldg_roof,main_bldg_year,main_bldg_effective_year
account_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
R049810025,1,130116,Residential,400 NW 43RD ST,,OKLAHOMA CITY,YOUNGS STEVEN ROGERS,TURLEY ALLISON,400 NW 43RD ST,,"OKLAHOMA CITY, OK 73118-8221",2661,,TXD 200,Oklahoma City #89,13068.0,None,None,52721,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,001,EDGEMERE TERRACE 001 001,1824.0,3.0,2.00,7.0,1824.0,1951.0,68.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Hip,1951.0,2014.0
R049810050,3,130117,Residential,408 NW 43RD ST,,OKLAHOMA CITY,JUNGMAN TIMOTHY WILLIAM & JULIE LYN,,408 NW 43RD ST,,"OKLAHOMA CITY, OK 73118-8221",2661,,TXD 200,Oklahoma City #89,9147.6,None,None,45900,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,002,EDGEMERE TERRACE 001 002,2677.0,3.0,2.50,6.0,2677.0,1951.0,68.0,Split Level,Central Air to Air,Fair,Frame Masonry Veneer,Hip,1951.0,2010.0
R049480080,6,130504,Residential,1009 NW 42ND ST,,OKLAHOMA CITY,SPEARS KENNETH L & REGINA,,210 NW 10TH ST,,"OKLAHOMA CITY, OK 73103-3902",2663,,TXD 200,Oklahoma City #89,7405.2,None,None,71460,Sect 16-T12N-R3W Qtr SW,CROWN HEIGHTS ADD,001,004,CROWN HEIGHTS ADD 001 004,1961.0,2.0,2.00,9.0,1961.0,1939.0,80.0,Duplex One Story,Central Air to Air,Fair,Frame Masonry Veneer,Hip,1939.0,1977.0
R049640005,7,131004,Residential,5021 N WALKER AVE,,OKLAHOMA CITY,CRISP STEVE & ANGIE,,5021 N WALKER,,"OKLAHOMA CITY, OK 73118",2664,,TXD 200,Oklahoma City #89,9147.6,None,None,48230,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 1 & 2,1352.0,3.0,1.50,6.0,1352.0,1951.0,68.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Gable,1951.0,1997.0
R049640010,8,131005,Residential,504 NW 50TH ST,,OKLAHOMA CITY,DICKSON JERRY ROBERT,,504 NW 50TH ST,,"OKLAHOMA CITY, OK 73118-6204",2664,,TXD 200,Oklahoma City #89,6969.6,None,None,37100,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 3 & 4,1386.0,3.0,1.50,6.0,1386.0,1951.0,68.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Hip,1951.0,2001.0
R049640015,9,131006,Residential,508 NW 50TH ST,,OKLAHOMA CITY,GARRETT & COMPANY LLC,,9701 BROADWAY EXT,,"OKLAHOMA CITY, OK 73114-6316",2664,,TXD 200,Oklahoma City #89,6969.6,None,None,37100,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 5 & 6,1092.0,2.0,1.00,4.0,1092.0,1950.0,69.0,Ranch 1 Story,Floor Wall Furnace,Fair,Frame Masonry Veneer,Hip,1950.0,1950.0
R049810100,10,130119,Residential,418 NW 43RD ST,,OKLAHOMA CITY,BRADEN THOMAS GARY & BEVERLY JOAN,REVOCABLE LIVING TRUST,418 NW 43RD ST,,"OKLAHOMA CITY, OK 73118-8221",2661,,TXD 200,Oklahoma City #89,9147.6,None,None,45900,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,004,EDGEMERE TERRACE 001 004,2047.0,2.0,2.00,7.0,2047.0,1951.0,68.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Hip,1951.0,1992.0
R049640020,12,131007,Residential,512 NW 50TH ST,,OKLAHOMA CITY,HALPIN JEFFREY J,,512 NW 50TH ST,,"OKLAHOMA CITY, OK 73118-6204",2664,,TXD 200,Oklahoma City #89,6969.6,None,None,37100,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 7 & 8,1038.0,2.0,1.00,4.0,1038.0,1950.0,69.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Gable,1950.0,1980.0
R049480100,13,130505,Residential,1005 NW 42ND ST,,OKLAHOMA CITY,DK INC,,C/O KENNETH L SPEARS,PO BOX 687,"OKLAHOMA CITY, OK 73101",2663,,TXD 200,Oklahoma City #89,7405.2,None,None,71460,Sect 16-T12N-R3W Qtr SW,CROWN HEIGHTS ADD,001,005,CROWN HEIGHTS ADD 001 005,1660.0,2.0,2.00,8.0,1660.0,1947.0,72.0,Duple

In [167]:
# This filters for subdivision based on number of properties in that sub. Let's change it
# later to filter for number of transactions.
sublist = (rp_df[['subdivision']]
            .assign(count = 0)
            .groupby("subdivision")
            .count())
sublist = sublist.where(sublist['count'] >= 100).dropna()
sublist_names = sublist.index.values
display(sublist_names)

rp_df['subdivision'] = rp_df.apply(lambda x: 'NOT LISTED' if x['subdivision'] not in sublist_names else x['subdivision'], axis=1)

array(['AURORA ADDITION', 'BAUMANS ADDITION', 'BELL VERN ADDITION',
       'CARLE & COLCORD ADDITION', 'CASHION PLACE ADDITION',
       'CASHION PLACE AMD', 'CENTRAL PARK ADDITION',
       'CLASSENS CREAM RIDGE', 'COLLEGE ADDITION', 'CRESTWOOD ADDITION',
       'CROWN HEIGHTS ADD', 'DARRALLS SECOND ADD', 'DENNISTON PARK ADD',
       'DOUGLAS PLACE ADD', 'EDGEMERE PARK ADD', 'EDGEMERE TERRACE',
       'EPWORTH VIEW ADD', 'GARDEN ADDITION', 'GATEWOOD ADDITION',
       'GRAND VIEW ADDITION', 'GUERNSEY PARK PLACE',
       'HEMINGWAY CONDOMINIUMS', 'INGLESIDE TO OKLA CITY',
       'JEFFERSON PARK ADD', 'LAS VEGAS ADDITION',
       'LINWOOD PLACE AMENDED', 'LINWOOD PLACE SECOND',
       'LYONS WILL ROGERS PK', 'MAYFAIR HEIGHTS BLKS 1 THRU 29',
       'MILAM PLACE ADDITION', 'MILITARY PARK ADD', 'MILLERS BOULEVARD',
       'MORRIS WILL ROGER PARK', 'NEAS ADDITION', 'PACKINGHOUSE PARK',
       'PARKER & COLCORD ADD', 'PUTNAM HEIGHTS 2ND', 'PUTNAM HEIGHTS ADD',
       'RAVENSWOOD ADDITION', 'RI

In [168]:
sales_df = pd.DataFrame(sales)
display(sales_df)

,account_number,date,days_ago,price
0,R188308300,2019-10-10,27,216000
1,R212841600,2019-10-10,27,227000
2,R109711175,2019-10-09,28,75000
3,R109731255,2019-10-09,28,120000
4,R151661260,2019-10-09,28,49000
5,R056651065,2019-10-09,28,65000
6,R122251185,2019-10-09,28,86000
7,R130762112,2019-10-08,29,40500
8,R203061190,2019-10-08,29,825000
9,R034354750,2019-10-08,29,140000


In [169]:
merged_df = pd.merge(rp_df, sales_df, on='account_number', how='inner')
display(merged_df)

,account_number,id,propertyid,property_type,location,building_name_occupant,city,owner_name_1,owner_name_2,billing_address_1,billing_address_2,city_state_zip,quarter_section,parent_acct,tax_district,school_system,land_size,lot_width,lot_depth,land_value,quarter_section_description,subdivision,block,lot,legal_description,sqft_sum,bed_sum,bath_sum,room_sum,main_bldg_sqft,year_built,bldg_age,main_bldg_description,main_bldg_hvac,main_bldg_quality,main_bldg_exterior,main_bldg_roof,main_bldg_year,main_bldg_effective_year,date,days_ago,price
0,R049640005,7,131004,Residential,5021 N WALKER AVE,,OKLAHOMA CITY,CRISP STEVE & ANGIE,,5021 N WALKER,,"OKLAHOMA CITY, OK 73118",2664,,TXD 200,Oklahoma City #89,9147.6,None,None,48230,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 1 & 2,1352.0,3.0,1.50,6.0,1352.0,1951.0,68.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Gable,1951.0,1997.0,2019-07-02,127,108000
1,R049640005,7,131004,Residential,5021 N WALKER AVE,,OKLAHOMA CITY,CRISP STEVE & ANGIE,,5021 N WALKER,,"OKLAHOMA CITY, OK 73118",2664,,TXD 200,Oklahoma City #89,9147.6,None,None,48230,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 1 & 2,1352.0,3.0,1.50,6.0,1352.0,1951.0,68.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Gable,1951.0,1997.0,2019-05-10,180,108000
2,R049810175,27,130122,Residential,436 NW 43RD ST,,OKLAHOMA CITY,BELFLOWER ADAM P & ALEXANDRA,,436 NW 43RD ST,,"OKLAHOMA CITY , OK 73118",2661,,TXD 200,Oklahoma City #89,9147.6,None,None,45900,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,001,007,EDGEMERE TERRACE 001 007,2018.0,3.0,2.00,7.0,2018.0,1951.0,68.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Hip,1951.0,2007.0,2019-04-05,215,305000
3,R049640070,50,131015,Residential,5000 N LEE AVE,,OKLAHOMA CITY,KUMBALEK MICHAEL MAX,,5000 N LEE AVE,,"OKLAHOMA CITY, OK 73118",2664,,TXD 200,Oklahoma City #89,6969.6,None,None,37100,Sect 16-T12N-R3W Qtr NW,DOUGLAS PLACE ADD,001,000,DOUGLAS PLACE ADD 001 000 LOTS 23 & 24,1429.0,2.0,1.75,6.0,1429.0,1950.0,69.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Gable,1950.0,1970.0,2019-05-15,175,157000
4,R049810375,66,130130,Residential,427 NW 43RD ST,,OKLAHOMA CITY,STALL JOSPEH E,LAWMASTER JENNIFER L,427 NW 43RD ST,,"OKLAHOMA CITY, OK 73118",2661,,TXD 200,Oklahoma City #89,8276.4,None,None,42840,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,002,005,EDGEMERE TERRACE 002 005,2155.0,3.0,2.00,4.0,2155.0,1951.0,68.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Hip,1951.0,2007.0,2019-09-06,61,335000
5,R049810425,76,130132,Residential,415 NW 43RD ST,,OKLAHOMA CITY,SCHEER BRADY D,SCHEER MACKENZIE K,415 NW 43RD ST,,"OKLAHOMA CITY, OK 73118",2661,,TXD 200,Oklahoma City #89,8276.4,None,None,42840,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,002,007,EDGEMERE TERRACE 002 007,2097.0,3.0,2.50,7.0,2097.0,1952.0,67.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Hip,1952.0,2010.0,2019-06-05,154,343500
6,R049810425,76,130132,Residential,415 NW 43RD ST,,OKLAHOMA CITY,SCHEER BRADY D,SCHEER MACKENZIE K,415 NW 43RD ST,,"OKLAHOMA CITY, OK 73118",2661,,TXD 200,Oklahoma City #89,8276.4,None,None,42840,Sect 16-T12N-R3W Qtr NE,EDGEMERE TERRACE,002,007,EDGEMERE TERRACE 002 007,2097.0,3.0,2.50,7.0,2097.0,1952.0,67.0,Ranch 1 Story,Central Air to Air,Fair,Frame Masonry Veneer,Hip,1952.0,2010.0,2019-05-06,184,343500
7,R049820200,94,130383,Residential,305 NW 42ND ST,,OKLAHOMA CITY,SCAPERLANDA MICHAEL A & MARIA RUIZ,,3816 WAVERLY CT,,"NORMAN, OK 73072",2662,,TXD 200,Oklahoma City #89,17859.6,None,None,85603,Sect 16-T12N-R3W Qtr SE,NOT LISTED,001,002,EDGEMERE HEIGHTS ADD 001 002,2541.0,4.0,2.00,8.0,2541.0,1952.0,67.0,Ranch 1 Story,Central Air to Air,Average,Frame Masonry Veneer,Hip,1952.0,2019.0,2019-05-29,161,415000
8,R049810550,101,130137,Residential,412 NW 44TH ST,,OKLAHOMA CITY,PENNINGTON PARKER B,,412 NW 44TH ST,,"OKLAHOMA CITY, OK 73118",2661,,TXD 200,Oklahoma City #89,7405.2,None,None,36720,Sect

In [170]:
included_fields = ['land_size', 'land_value', 'subdivision', 'bldg_age', 'year_built', 'sqft_sum', 'bed_sum', 'bath_sum', 'price', 'days_ago', 'main_bldg_hvac', 'main_bldg_description', 'main_bldg_exterior', 'main_bldg_quality', 'main_bldg_roof', 'main_bldg_year', 'main_bldg_effective_year', 'room_sum']
new_df = merged_df[included_fields]

display(new_df)

,land_size,land_value,subdivision,bldg_age,year_built,sqft_sum,bed_sum,bath_sum,price,days_ago,main_bldg_hvac,main_bldg_description,main_bldg_exterior,main_bldg_quality,main_bldg_roof,main_bldg_year,main_bldg_effective_year,room_sum
0,9147.6,48230,DOUGLAS PLACE ADD,68.0,1951.0,1352.0,3.0,1.50,108000,127,Central Air to Air,Ranch 1 Story,Frame Masonry Veneer,Fair,Gable,1951.0,1997.0,6.0
1,9147.6,48230,DOUGLAS PLACE ADD,68.0,1951.0,1352.0,3.0,1.50,108000,180,Central Air to Air,Ranch 1 Story,Frame Masonry Veneer,Fair,Gable,1951.0,1997.0,6.0
2,9147.6,45900,EDGEMERE TERRACE,68.0,1951.0,2018.0,3.0,2.00,305000,215,Central Air to Air,Ranch 1 Story,Frame Masonry Veneer,Fair,Hip,1951.0,2007.0,7.0
3,6969.6,37100,DOUGLAS PLACE ADD,69.0,1950.0,1429.0,2.0,1.75,157000,175,Central Air to Air,Ranch 1 Story,Frame Masonry Veneer,Fair,Gable,1950.0,1970.0,6.0
4,8276.4,42840,EDGEMERE TERRACE,68.0,1951.0,2155.0,3.0,2.00,335000,61,Central Air to Air,Ranch 1 Story,Frame Masonry Veneer,Fair,Hip,1951.0,2007.0,4.0
5,8276.4,42840,EDGEMERE TERRACE,67.0,1952.0,2097.0,3.0,2.50,343500,154,Central Air to Air,Ranch 1 Story,Frame Masonry Veneer,Fair,Hip,1952.0,2010.0,7.0
6,8276.4,42840,EDGEMERE TERRACE,67.0,1952.0,2097.0,3.0,2.50,343500,184,Central Air to Air,Ranch 1 Story,Frame Masonry Veneer,Fair,Hip,1952.0,2010.0,7.0
7,17859.6,85603,NOT LISTED,67.0,1952.0,2541.0,4.0,2.00,415000,161,Central Air to Air,Ranch 1 Story,Frame Masonry Veneer,Average,Hip,1952.0,2019.0,8.0
8,7405.2,36720,EDGEMERE TERRACE,68.0,1951.0,1416.0,3.0,1.75,182500,237,Central Air to Air,Ranch 1 Story,Frame Masonry Veneer,Fair,Hip,1951.0,2015.0,5.0
9,8276.4,80559,CROWN HEIGHTS ADD,83.0,1936.0,2316.0,4.0,1.75,555000,146,Central Air to Air,1½ Story Fin,Frame Masonry Veneer,Average,Gable,1936.0,2017.0,9.0


In [171]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

### One Hot Encoding

We use one hot encoding for the subdivision feature. As a result, we create a feature for each individual neighborhood and other categorical features.

In [172]:
from sklearn.linear_model import LinearRegression
new_df = (merged_df[included_fields][merged_df.price != 0]
          .dropna()
          .reset_index()
          .drop(columns=['index']))
#new_df = new_df[new_df.price != 0]

ohenc = preprocessing.OneHotEncoder()

In [173]:
def return_encoded(df, feature_name, feature_prefix):
    ohenc.fit([[x] for x in df[feature_name]])
    encoded = ohenc.transform([[x] for x in df[feature_name]]).toarray()
    return pd.DataFrame(encoded, columns=[feature_prefix+"-" + ohenc.categories_[0][i] for i in range(encoded.shape[1])])

In [174]:
encoded_df_names = [
    ['subdivision','Sub'],
    ['main_bldg_hvac', 'HVAC'],
    ['main_bldg_description', 'Descr'],
    ['main_bldg_exterior', 'Exterior'],
    ['main_bldg_roof', 'Roof']
]

encoded_dfs = []
for (col_name, prefix) in encoded_df_names:
    cur_df = return_encoded(new_df, col_name, prefix)
    encoded_dfs.append(cur_df)

In [175]:
unused_col_names = ['main_bldg_quality']

new_df = new_df.drop(columns=([x[0] for x in encoded_df_names]+[x for x in unused_col_names]))
new_df = pd.concat(([new_df] + [x for x in encoded_dfs]), axis=1)
display(new_df)

,land_size,land_value,bldg_age,year_built,sqft_sum,bed_sum,bath_sum,price,days_ago,main_bldg_year,main_bldg_effective_year,room_sum,Sub-AURORA ADDITION,Sub-BAUMANS ADDITION,Sub-BELL VERN ADDITION,Sub-CARLE & COLCORD ADDITION,Sub-CASHION PLACE ADDITION,Sub-CASHION PLACE AMD,Sub-CENTRAL PARK ADDITION,Sub-CLASSENS CREAM RIDGE,Sub-COLLEGE ADDITION,Sub-CRESTWOOD ADDITION,Sub-CROWN HEIGHTS ADD,Sub-DARRALLS SECOND ADD,Sub-DENNISTON PARK ADD,Sub-DOUGLAS PLACE ADD,Sub-EDGEMERE PARK ADD,Sub-EDGEMERE TERRACE,Sub-EPWORTH VIEW ADD,Sub-GARDEN ADDITION,...,Descr-Duplex One half Story,Descr-Duplex Two Story,Descr-Permit Value,Descr-Ranch 1 Story,Descr-Salvage,Descr-Split Level,Descr-Townhouse Multiple Story,Descr-Triplex Multiple Story,Exterior-,Exterior-Frame Aluminum,Exterior-Frame Asbestos,Exterior-Frame Brick Veneer,Exterior-Frame Hardboard,Exterior-Frame Masonry Veneer,Exterior-Frame Permastone,Exterior-Frame Siding,Exterior-Frame Stucco,Exterior-Frame Vinyl,Exterior-High Profile Dome,Exterior-Masonry Concrete Block,Exterior-Masonry Stone,Exterior-Minimal,Exterior-None,Roof-Flat,Roof-Gable,Roof-Gable/Hip,Roof-Gambrel,Roof-Hip,Roof-Hip/Gable,Roof-Mansard
0,9147.6,48230,68.0,1951.0,1352.0,3.0,1.50,108000,127,1951.0,1997.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,9147.6,48230,68.0,1951.0,1352.0,3.0,1.50,108000,180,1951.0,1997.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,9147.6,45900,68.0,1951.0,2018.0,3.0,2.00,305000,215,1951.0,2007.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,6969.6,37100,69.0,1950.0,1429.0,2.0,1.75,157000,175,1950.0,1970.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,8276.4,42840,68.0,1951.0,2155.0,3.0,2.00,335000,61,1951.0,2007.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,8276.4,42840,67.0,1952.0,2097.0,3.0,2.50,343500,154,1952.0,2010.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,8276.4,42840,67.0,1952.0,2097.0,3.0,2.50,343500,184,1952.0,2010.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,17859.6,85603,67.0,1952.0,2541.0,4.0,2.00,415000,161,1952.0,2019.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,7405.2,36720,68.0,1951.0,1416.0,3.0,1.75,182500,237,1951.0,2015.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,8276.4,80559,83.0,1936.0,2316.0,4.0,1.75,555000,146,1936.0,2017.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Correlation

Let's look at the correlation of the various featues. We see that some features like square footage, # of beds/baths, and land size have a positive correlation.

Some of the more desirable subdivisions have a positive correlation too, while many other neighborhoods have a negative correlation with the sale price (relative to other sales in our data set). Do note that this is correlation for the price of the property - not price relative to building or land size.

Year built/building age is close enough to zero to have no correlation. And as we might expect, days_ago has a negative correlation, indicating that prices have generally been increasing over time.

In [176]:
pd.set_option('max_rows',100)
dfcorr = new_df.corr()
new_df

,land_size,land_value,bldg_age,year_built,sqft_sum,bed_sum,bath_sum,price,days_ago,main_bldg_year,main_bldg_effective_year,room_sum,Sub-AURORA ADDITION,Sub-BAUMANS ADDITION,Sub-BELL VERN ADDITION,Sub-CARLE & COLCORD ADDITION,Sub-CASHION PLACE ADDITION,Sub-CASHION PLACE AMD,Sub-CENTRAL PARK ADDITION,Sub-CLASSENS CREAM RIDGE,Sub-COLLEGE ADDITION,Sub-CRESTWOOD ADDITION,Sub-CROWN HEIGHTS ADD,Sub-DARRALLS SECOND ADD,Sub-DENNISTON PARK ADD,Sub-DOUGLAS PLACE ADD,Sub-EDGEMERE PARK ADD,Sub-EDGEMERE TERRACE,Sub-EPWORTH VIEW ADD,Sub-GARDEN ADDITION,...,Descr-Duplex One half Story,Descr-Duplex Two Story,Descr-Permit Value,Descr-Ranch 1 Story,Descr-Salvage,Descr-Split Level,Descr-Townhouse Multiple Story,Descr-Triplex Multiple Story,Exterior-,Exterior-Frame Aluminum,Exterior-Frame Asbestos,Exterior-Frame Brick Veneer,Exterior-Frame Hardboard,Exterior-Frame Masonry Veneer,Exterior-Frame Permastone,Exterior-Frame Siding,Exterior-Frame Stucco,Exterior-Frame Vinyl,Exterior-High Profile Dome,Exterior-Masonry Concrete Block,Exterior-Masonry Stone,Exterior-Minimal,Exterior-None,Roof-Flat,Roof-Gable,Roof-Gable/Hip,Roof-Gambrel,Roof-Hip,Roof-Hip/Gable,Roof-Mansard
0,9147.6,48230,68.0,1951.0,1352.0,3.0,1.50,108000,127,1951.0,1997.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,9147.6,48230,68.0,1951.0,1352.0,3.0,1.50,108000,180,1951.0,1997.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,9147.6,45900,68.0,1951.0,2018.0,3.0,2.00,305000,215,1951.0,2007.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,6969.6,37100,69.0,1950.0,1429.0,2.0,1.75,157000,175,1950.0,1970.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,8276.4,42840,68.0,1951.0,2155.0,3.0,2.00,335000,61,1951.0,2007.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,8276.4,42840,67.0,1952.0,2097.0,3.0,2.50,343500,154,1952.0,2010.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,8276.4,42840,67.0,1952.0,2097.0,3.0,2.50,343500,184,1952.0,2010.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,17859.6,85603,67.0,1952.0,2541.0,4.0,2.00,415000,161,1952.0,2019.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,7405.2,36720,68.0,1951.0,1416.0,3.0,1.75,182500,237,1951.0,2015.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,8276.4,80559,83.0,1936.0,2316.0,4.0,1.75,555000,146,1936.0,2017.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [177]:
pd.set_option('max_rows',200)

display(dfcorr['price'].sort_values())

Descr-Ranch 1 Story                  -0.345224
HVAC-Floor Wall Furnace              -0.325195
Roof-Gable                           -0.150675
Exterior-Frame Vinyl                 -0.146219
Exterior-Frame Asbestos              -0.140205
Sub-NEAS ADDITION                    -0.088394
Sub-BAUMANS ADDITION                 -0.084138
Exterior-Frame Hardboard             -0.082175
Sub-WEST LAWN GARDEN ADD             -0.082135
Sub-LYONS WILL ROGERS PK             -0.079698
Sub-NOT LISTED                       -0.075604
Sub-SHARTEL BOULEVARD                -0.071500
Descr-Condo <= 3 Stories             -0.070712
Sub-WEST POINT AMD E2                -0.067853
Sub-RAVENSWOOD ADDITION              -0.065427
Sub-CARLE & COLCORD ADDITION         -0.062207
Sub-PARKER & COLCORD ADD             -0.059808
Descr-Duplex One Story               -0.059497
Sub-HEMINGWAY CONDOMINIUMS           -0.058515
Exterior-Minimal                     -0.057605
Sub-INGLESIDE TO OKLA CITY           -0.056546
Sub-AURORA AD

In [178]:
new_df = new_df.drop('bldg_age',axis=1)

### Creating a linear regression model

Let's create our linear regression model.

In [179]:
X = new_df.drop('price',axis=1)
y = new_df['price']
#display(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
linreg = LinearRegression().fit(X_train, y_train)
#display(X_test)
linreg.score(X_test, y_test)

0.7897173502089017

In [180]:
'''
import numpy as np
ch_oh = ohenc.transform([["EDGEMERE TERRACE"]]).toarray()
ch_args = [10000, 40000, 1990, 3000, 4, 3, 10] + ch_oh.tolist()[0]
ch_args = [6400, 34790, 1500, 1990, 4, 3, 10] + ch_oh.tolist()[0]
#ch_args = ch_oh.tolist()
linreg.predict([ch_args])
'''
linreg.score(X_train,y_train)

0.8369230230531489